In [ ]:
import astropy.coordinates as coord
from astropy.io import fits
from astropy.table import Table, vstack
import astropy.units as u
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from pyia import GaiaData
import gala.coordinates as gc
from scipy.ndimage import gaussian_filter
from scipy.stats import binned_statistic_2d
from gala.mpl_style import hesperia, laguna, center_emph, center_deemph
from tqdm import tqdm
import dustmaps
from dustmaps.sfd import SFDQuery

In [ ]:
# import dustmaps.sfd
# dustmaps.sfd.fetch()

In [ ]:
L_lim = [-140, 120] * u.deg
B_lim = [-60, 60] * u.deg

In [ ]:
import scipy.stats

def get_ext(G, bp, rp, ebv, maxnit=10):
    """ Compute the Gaia extinctions assuming relations from Babusieux
    Arguments: G, bp, rp, E(B-V)
    maxnit -- number of iterations
    Returns extinction in G,bp, rp
    Author: Sergey Koposov skoposov@cmu.edu
    """
    c1, c2, c3, c4, c5, c6, c7 = [0.9761, -0.1704,
                                  0.0086, 0.0011, -0.0438, 0.0013, 0.0099]
    d1, d2, d3, d4, d5, d6, d7 = [
        1.1517, -0.0871, -0.0333, 0.0173, -0.0230, 0.0006, 0.0043]
    e1, e2, e3, e4, e5, e6, e7 = [
        0.6104, -0.0170, -0.0026, -0.0017, -0.0078, 0.00005, 0.0006]
    A0 = 3.1*ebv
    P1 = np.poly1d([c1, c2, c3, c4][::-1])

    def F1(bprp): return np.poly1d(
        [c1, c2, c3, c4][::-1])(bprp)+c5*A0+c6*A0**2+c7*bprp*A0

    def F2(bprp): return np.poly1d(
        [d1, d2, d3, d4][::-1])(bprp)+d5*A0+d6*A0**2+d7*bprp*A0

    def F3(bprp): return np.poly1d(
        [e1, e2, e3, e4][::-1])(bprp)+e5*A0+e6*A0**2+e7*bprp*A0
    xind = np.isfinite(bp+rp+G)
    curbp = bp-rp
    for i in range(maxnit):
        AG = F1(curbp)*A0
        Abp = F2(curbp)*A0
        Arp = F3(curbp)*A0
        curbp1 = bp-rp-Abp+Arp
        delta = np.abs(curbp1-curbp)[xind]
        print(scipy.stats.scoreatpercentile(delta, 99))
        curbp = curbp1
    AG = F1(curbp)*A0
    Abp = F2(curbp)*A0
    Arp = F3(curbp)*A0
    return AG, Abp, Arp

In [ ]:
def reflex(c):
    c = coord.SkyCoord(c)
    # Correct for reflex motion
    v_sun = coord.Galactocentric.galcen_v_sun
    observed = c.transform_to(coord.Galactic)
    rep = observed.cartesian.without_differentials()
    rep = rep.with_differentials(observed.cartesian.differentials['s'] + v_sun)
    return coord.Galactic(rep).transform_to(c.frame)

In [ ]:
# M_G = 2*u.mag
M_G = -1*u.mag
coord.Distance(30*u.kpc).distmod + M_G

### SFD map of region

In [ ]:
LL = np.arange(L_lim[0].value, L_lim[1].value+0.1, 0.1)
BB = np.arange(B_lim[0].value, B_lim[1].value+0.1, 0.1)
LL, BB = np.meshgrid(LL, BB)
coords = coord.SkyCoord(LL*u.deg, BB*u.deg,
                        distance=1.*u.kpc, frame=gc.MagellanicStream).transform_to(coord.Galactic)

In [ ]:
sfd = SFDQuery()
EBV_grid_sfd = sfd(coords)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))

c = ax.pcolormesh(LL, BB, 
   EBV_grid_sfd,
   cmap='binary',
   norm=mpl.colors.LogNorm(vmin=1E-2, vmax=1e0)
)

ax.set_xlim(120, -140)
ax.set_ylim(-60, 60)

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$B_{\rm MS}$ [deg]')
fig.tight_layout()

fig.savefig('sfd_mag.png', dpi=250)

BP-RP and G mag cuts to get ~A/B stars @ 30 kpc

In [ ]:
g = GaiaData('/Users/adrian/data/GaiaDR2/blue_distant.fits')
g = g[g.parallax < 0.5*u.mas]
len(g)

In [ ]:
c = coord.SkyCoord(ra=g.ra, dec=g.dec,
                   distance=np.full(len(g), 35)*u.kpc,
                   pm_ra_cosdec=g.pmra,
                   pm_dec=g.pmdec,
                   radial_velocity=np.zeros(len(g))*u.km/u.s)

In [ ]:
EBV = sfd(c)

In [ ]:
G = g.phot_g_mean_mag.value
bp = g.phot_bp_mean_mag.value
rp = g.phot_rp_mean_mag.value
A_G, A_BP, A_RP = get_ext(G, bp, rp, EBV)

In [ ]:
ext_mask = np.isfinite(A_G) & np.isfinite(A_BP) & np.isfinite(A_RP) & (A_G < 4)
ext_mask.sum()

In [ ]:
mag_c = c.transform_to(gc.MagellanicStream)
mag_c = reflex(mag_c)

In [ ]:
L = mag_c.L.wrap_at(180*u.deg)
B = mag_c.B
mag_mask = ((L > L_lim[0]) & (L < L_lim[1]) &
            (B > B_lim[0]) & (B < B_lim[1]) & 
            (np.sqrt(L**2 + (B-4*u.deg)**2) > 10*u.deg) & # Remove LMC
            (np.sqrt((L+10*u.deg)**2 + (B+6*u.deg)**2) > 4*u.deg) & # Remove bridge
            (np.sqrt((L+15*u.deg)**2 + (B+11*u.deg)**2) > 5*u.deg) # Remove SMC
           )

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))

ax.plot(mag_c.L.wrap_at(180*u.deg).degree[ext_mask & mag_mask], 
        mag_c.B.degree[ext_mask & mag_mask], 
        linestyle='none', marker='.', alpha=0.1)

ax.set_xlim(120, -140)
ax.set_ylim(-60, 60)

In [ ]:
bprp0 = (g.phot_bp_mean_mag - g.phot_rp_mean_mag) - (A_BP - A_RP)*u.mag
G0 = g.phot_g_mean_mag - A_G*u.mag

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(bprp0, G0, marker=',', linestyle='none', alpha=0.2)

ax.set_xlim(-2, 0.5)
ax.set_ylim(20, 12)

In [ ]:
# more_mask = (G0 > 16.*u.mag) & (bprp0 > -0.5*u.mag)
# more_mask = (bprp0 < -0*u.mag) & (bprp0 > -0.5*u.mag)
more_mask = (bprp0 < 0.3*u.mag) & (bprp0 > -0.5*u.mag) & (G0 > (2.5/0.5*bprp0 + 17.5*u.mag))

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(bprp0, G0, marker=',', linestyle='none', alpha=0.2)
ax.plot(bprp0[more_mask], G0[more_mask], marker=',', linestyle='none', alpha=1)

ax.set_xlim(-2, 0.5)
ax.set_ylim(12, 20)

In [ ]:
all_mask = mag_mask & ext_mask & more_mask

In [ ]:
# pm_mask = (mag_c.pm_B > -0.1*u.mas/u.yr) & (mag_c.pm_B < 3*u.mas/u.yr)
# pm_mask = np.abs(mag_c.pm_B) < 0.6*u.mas/u.yr
# pm_mask = np.ones(len(mag_c)).astype(bool)
# pm_mask = np.sqrt((mag_c.pm_L_cosB - -2.3*u.mas/u.yr)**2 + (mag_c.pm_B - -1.5*u.mas/u.yr)**2) < 0.5*u.mas/u.yr

# left_mask = np.abs(sag_c.Beta) < 10*u.deg
left_mask = (((mag_c.B > 30*u.deg) & (mag_c.B < 60*u.deg) &
              (mag_c.L.wrap_at(180*u.deg) > -110*u.deg) & (mag_c.L.wrap_at(180*u.deg) < -20*u.deg)) |
             ((mag_c.B > -60*u.deg) & (mag_c.B < -20*u.deg) &
              (mag_c.L.wrap_at(180*u.deg) > 50*u.deg) & (mag_c.L.wrap_at(180*u.deg) < 100*u.deg)))

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)

axes[0].plot(mag_c.pm_L_cosB[all_mask], mag_c.pm_B[all_mask], 
             linestyle='none', marker='.', alpha=0.2)

axes[1].plot(mag_c.pm_L_cosB[left_mask], mag_c.pm_B[left_mask], 
             linestyle='none', marker='.', alpha=0.2)

# axes[2].plot(mag_c.pm_L_cosB[all_mask & pm_mask], mag_c.pm_B[all_mask & pm_mask], 
#              linestyle='none', marker=',', alpha=0.75)

# ax.set_xlim(90, -120)
# ax.set_ylim(-40, 20)

In [ ]:
pm_bins = np.arange(-5, 5+0.1, 0.1)

bg_pm, pm_x, pm_y = np.histogram2d(mag_c.pm_L_cosB[left_mask].value, 
                                   mag_c.pm_B[left_mask].value,
                                   bins=(pm_bins, pm_bins))

fg_pm, pm_x, pm_y = np.histogram2d(mag_c.pm_L_cosB[all_mask].value, 
                                   mag_c.pm_B[all_mask].value,
                                   bins=(pm_bins, pm_bins))

fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)

axes[0].pcolormesh(pm_x, pm_y, fg_pm.T)
axes[1].pcolormesh(pm_x, pm_y, bg_pm.T)
axes[2].pcolormesh(pm_x, pm_y, fg_pm.T/31200. - bg_pm.T/4700.)

r = mpl.patches.Circle((0.7, 0.1), radius=1, facecolor='none', edgecolor='k')
axes[2].add_patch(r)

fig.tight_layout()

In [ ]:
pm_mask = np.sqrt((mag_c.pm_L_cosB - 0.7*u.mas/u.yr)**2 + 
                  (mag_c.pm_B - 0.1*u.mas/u.yr)**2) < 1*u.mas/u.yr

In [ ]:
ol_blob = coord.SkyCoord(ra=178.94197852110662*u.deg,
                         dec=-29.46957451372061*u.deg)
ol_blob = ol_blob.transform_to(gc.MagellanicStream)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))

H, xe, ye = np.histogram2d(mag_c.L.wrap_at(180*u.deg).degree[all_mask & pm_mask], 
                           mag_c.B.degree[all_mask & pm_mask],
                           bins=(np.arange(-140, 120+1, 1),
                                 np.arange(-60, 60+1, 1)))
H = gaussian_filter(H, sigma=0.7)
ax.pcolormesh(xe, ye, H.T, 
              norm=mpl.colors.LogNorm(vmin=0.3, vmax=6),
              cmap='Greys')

gas = [[(35, 5), (30, 5)],
       [(35, -17), (30, 7)],
       [(10, -32), (20, 17)],
       [(-35, -25), (15, 35)],
       [(-110, -10), (80, 20)]]
# for row in gas:
#     r = mpl.patches.Rectangle(row[0], row[1][0], row[1][1],
#                               facecolor='none', edgecolor='tab:red')
#     ax.add_patch(r)

r = mpl.patches.Circle((ol_blob.L.degree, ol_blob.B.degree), radius=2., 
                       facecolor='none', edgecolor='tab:green')
ax.add_patch(r)
    
ax.set_xlim(120, -140)
ax.set_ylim(-60, 60)

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$B_{\rm MS}$ [deg]')

fig.tight_layout()

fig.savefig('mag_blue.png', dpi=250)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))

ax.plot(mag_c.L.wrap_at(180*u.deg).degree[all_mask & pm_mask], 
        mag_c.B.degree[all_mask & pm_mask],
        marker='.', linestyle='none', alpha=0.5)

gas = [[(35, 5), (30, 5)],
       [(35, -17), (30, 7)],
       [(10, -32), (20, 17)],
       [(-35, -25), (15, 35)],
       [(-110, -10), (80, 20)]]
for row in gas:
    r = mpl.patches.Rectangle(row[0], row[1][0], row[1][1],
                              facecolor='none', edgecolor='tab:red')
    ax.add_patch(r)

r = mpl.patches.Circle((ol_blob.L.degree, ol_blob.B.degree), radius=2., 
                       facecolor='none', edgecolor='tab:green')
ax.add_patch(r)
    
ax.set_xlim(120, -140)
ax.set_ylim(-60, 60)

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$B_{\rm MS}$ [deg]')

# fig.savefig('mag_blue.png', dpi=250)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))

ax.plot(c.ra.degree[all_mask & pm_mask], 
        c.dec.degree[all_mask & pm_mask],
        marker='.', linestyle='none', alpha=0.5)
    
ax.set_xlim(360, 0)
# ax.set_ylim(-60, 60)

# ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
# ax.set_ylabel(r'$B_{\rm MS}$ [deg]')

In [ ]:
stat_muL = binned_statistic_2d(mag_c.L.wrap_at(180*u.deg).degree[all_mask & pm_mask],
                               mag_c.B.degree[all_mask & pm_mask],
                               mag_c.pm_L_cosB[all_mask & pm_mask],
                               bins=(np.arange(-140, 120+1, 2),
                                     np.arange(-60, 60+1, 2)),
                               statistic=np.nanmean)

stat_muB = binned_statistic_2d(mag_c.L.wrap_at(180*u.deg).degree[all_mask & pm_mask],
                               mag_c.B.degree[all_mask & pm_mask],
                               mag_c.pm_B[all_mask & pm_mask],
                               bins=(np.arange(-140, 120+1, 2),
                                     np.arange(-60, 60+1, 2)),
                               statistic=np.nanmean)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))

# H = gaussian_filter(H, sigma=0.75)
cc = ax.pcolormesh(stat_muL.x_edge, stat_muL.y_edge, stat_muL.statistic.T,
                   vmin=-2, vmax=2, cmap=center_emph)
cb = fig.colorbar(cc)
cb.set_label(r'${\rm mean}(\mu_L \cos B)$')

ax.set_xlim(120, -140)
ax.set_ylim(-60, 60)

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$B_{\rm MS}$ [deg]')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))

# H = gaussian_filter(H, sigma=0.75)
cc = ax.pcolormesh(stat_muB.x_edge, stat_muB.y_edge, stat_muB.statistic.T,
                   vmin=-1, vmax=1, cmap=center_emph)
cb = fig.colorbar(cc)
cb.set_label(r'${\rm mean}(\mu_B)$')

ax.set_xlim(120, -140)
ax.set_ylim(-60, 60)

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$B_{\rm MS}$ [deg]')

fig.tight_layout()
# fig.savefig('mag_pm.png', dpi=250)